# DS 2010 Final Project

## Import Packages

In [1]:
import pandas as pd
import math
import numpy as np
import datetime as dt

## Import Data

In [2]:
covid_19_data = pd.read_csv("Raw_Data/all-states-history-correct-range.csv")

anxiety_depression_data = pd.read_csv("Raw_Data/Indicators_of_Anxiety_or_Depression_Based_on_Reported_Frequency_of_Symptoms_During_Last_7_Days.csv")
# Starts Apr 23, Ends Nov 9

trips_data = pd.read_csv("Raw_Data/Trips_by_Distance.csv")
# trips_data2 = pd.read_csv("Raw_Data/Trips_by_Distance2.csv")

## Clean trip data

In [60]:
td = trips_data.groupby(by = ['Date', 'State Postal Code']).sum().drop(columns= ['State FIPS', 'County FIPS']).reset_index()
td['Week'] = np.floor(td.index / 51 / 7)
td2 = td.groupby(by = ['Week', 'State Postal Code']).sum().reset_index()
td2

,Week,State Postal Code,Population Staying at Home,Population Not Staying at Home,Number of Trips,Number of Trips <1,Number of Trips 1-3,Number of Trips 3-5,Number of Trips 5-10,Number of Trips 10-25,Number of Trips 25-50,Number of Trips 50-100,Number of Trips 100-250,Number of Trips 250-500,Number of Trips >=500
0,0.0,AK,3220673.0,7082591.0,2.897419e+07,8992772.0,8231687.0,3635131.0,3827901.0,2736117.0,1083549.0,316497.0,98020.0,15715.0,36802.0
1,0.0,AL,14561732.0,53868462.0,1.934874e+08,43540272.0,49001370.0,25808634.0,31077134.0,29209804.0,9846620.0,3411602.0,1273558.0,265330.0,53040.0
2,0.0,AR,9047480.0,33146070.0,1.194944e+08,27916370.0,31271648.0,15000080.0,18399048.0,17282118.0,6057438.0,2394556.0,950380.0,189272.0,33490.0
3,0.0,AZ,28810262.0,71592782.0,2.514977e+08,72521122.0,60440296.0,28613068.0,34635610.0,36573306.0,12008498.0,4024546.0,2123438.0,386488.0,171346.0
4,0.0,CA,161075465.0,392723165.0,1.275602e+09,340797705.0,335866619.0,144512367.0,174459323.0,176019070.0,68268533.0,24249859.0,8750564.0,1991002.0,686924.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1474,28.0,VT,3112011.0,8161371.0,3.335343e+07,6354524.0,8685732.0,4231276.0,4827171.0,5648798.0,2611117.0,724701.0,219875.0,40951.0,9283.0
1475,28.0,WA,40022575.0,95618063.0,3.629090e+08,86031010.0,90566383.0,42190273.0,55452475.0,57979032.0,21177510.0,6530974.0,2312758.0,401626.0,266979.0
1476,28.0,WI,29082135.0,75562089.0,3.055331e+08,65971931.0,80193353.0,37947144.0,46006069.0,45231781.0,17899922.0,8094189.0,3517971.0,550479.0,120279.0
1477,28.0,WV,9088279.0,23416697.0,9.196436e+07,18324623.0,24856511.0,11423885.0,13832736.0,13681066.0,5852528.0,2931675.0,911250.0,131684.0,18401.0


# Clean COVID-19 Data

In [5]:
# KEEP:
# DC: District of Columbia

# REMOVE:
# AS: American Samoa
# PR: Puerto Rico
# GU: Guam
# MP: Northern Mariana Islands
# VI: US Virgin Islands
remove_states = ['AS', 'PR', 'GU', 'MP', 'VI']

## prepare covid data

In [6]:
valuable_cols = ['state', 'date', 'death', 'deathConfirmed', 
                 'hospitalizedCurrently', 'positiveCasesViral', 
                 'positiveIncrease', 'totalTestsPeopleViral', 
                 'totalTestsPeopleViralIncrease', 'totalTestsViral']

In [7]:
# Remove undesired columns
covid_data = covid_19_data.copy()[valuable_cols]
# Convert string to datetime
covid_data['date'] = covid_data['date'].astype('datetime64[ns]')
# Convert target_ranges to list of 

### create Time Period column

In [8]:
ranges_list = ['4/23/2020-5/5/2020', '5/7/2020-5/12/2020', '5/14/2020-5/19/2020', '5/21/2020-5/26/2020',
               '5/28/2020-6/2/2020', '6/4/2020-6/9/2020', '6/11/2020-6/16/2020', '6/18/2020-6/23/2020',
               '6/25/2020-6/30/2020', '7/2/2020-7/7/2020', '7/9/2020-7/14/2020', '7/16/2020-7/21/2020',                  
               '8/19/2020-8/31/2020', '9/2/2020-9/14/2020', '9/16/2020-9/28/2020', '9/30/2020-10/12/2020', 
               '10/14/2020-10/26/2020', '10/28/2020-11/9/2020']

# Convert ranges_list to a list of lists of start and end datetimes
period_list = []
for r in ranges_list:
    r_list = []
    for date_str in r.split('-'):
        date = dt.datetime.strptime(date_str, '%m/%d/%Y').date()
        r_list.append(date)
    period_list.append(r_list)
    
# Create period column
time_periods = [] # -1: before first period | -2: after last period | 0-n: respective period
for index, row in covid_data.iterrows():
    true_period = np.NaN
    if row['date'] < period_list[0][0]:
        true_period = -1
    elif row['date'] > period_list[-1][-1]:
        true_period = -2
    else:
        for period in period_list:
            if period[0] <= row['date'] <= period[1]:
                true_period = period_list.index(period) + 1
    time_periods.append(true_period)

# Add Time Period column to covid_data
covid_data['Time Period'] = time_periods

# Make a copy before cleaning
data = covid_data.copy()

# Remove rows with invalid time periods
data.drop(data.loc[data['Time Period'] == -1].index, inplace=True)
data.drop(data.loc[data['Time Period'] == -2].index, inplace=True)
data.drop(data.loc[data['Time Period'] == np.NaN].index, inplace=True)

In [9]:
states = list(set(data['state'].values))
final_states = [item for item in states if item not in remove_states]

### Fill NaN and groupby Time Period for each state

In [54]:
clean_data = []
for state in final_states:
    state_data = data.loc[data['state'] == state]
    # TODO fill NaN by state
    state_data = state_data.groupby(['Time Period', 'state']).mean().reset_index()
    clean_data.append(state_data)
clean_df = pd.concat(clean_data)

clean_df["death"] = clean_df[["death", "deathConfirmed"]].max(axis=1)
clean_df["positiveCasesViral"] = clean_df[["positiveCasesViral", "totalTestsViral", "totalTestsPeopleViral"]].max(axis=1)
clean_df = clean_df.drop(columns=["deathConfirmed", "totalTestsViral", "totalTestsPeopleViral"])
clean_df.isnull().sum().sum()

54

In [62]:
clean_df[clean_df["hospitalizedCurrently"].isnull()]

,Time Period,state,death,hospitalizedCurrently,positiveCasesViral,positiveIncrease,totalTestsPeopleViralIncrease
0,1.0,UT,44.923077,NaN,1.154622e+05,154.153846,3789.461538
0,1.0,KS,124.923077,NaN,NaN,249.769231,0.000000
1,2.0,KS,154.833333,NaN,NaN,230.333333,0.000000
2,3.0,KS,171.000000,NaN,6.262867e+04,145.333333,2142.833333
3,4.0,KS,184.833333,NaN,7.706300e+04,113.166667,1699.500000
4,5.0,KS,210.500000,NaN,9.129767e+04,112.333333,-14055.500000
5,6.0,KS,231.666667,NaN,NaN,80.000000,0.000000
6,7.0,KS,243.166667,NaN,1.320598e+05,101.166667,22790.166667
7,8.0,KS,254.500000,NaN,1.489445e+05,130.666667,2147.000000
8,9.0,KS,265.500000,NaN,1.694630e+05,245.500000,2485.000000
